In [1]:
a = open('/tmp/t3.precrypt', 'rb').read()

In [27]:
import zlib
import base64

In [5]:
c = zlib.decompress(a[8:])

In [7]:
len(c)

3000

In [300]:
bd.contain([112.0, 33])

True

In [17]:
import hexdump
import struct
import inspect

In [336]:
import socket
import random

sk = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
buf = struct.pack('HHHH', random.randint(0,255*255),random.randint(0,255*255),
                  random.randint(0,255*255),random.randint(0,255*255))
addr = ('localhost', 7878)
sk.sendto(buf, addr)
resp,_ = sk.recvfrom(1024)
hexdump.hexdump(resp)

00000000: 02 00 00 00 03 DF 2B 1B  20 07 25 A0 36 BB 28 D3  ......+. .%.6.(.
00000010: C8 DA EA D9 6E 82 7A 33  62 85 15 11 F4 9E 57 F2  ....n.z3b.....W.
00000020: FA F3 31 EA 63 00 0B 23  92 1B 62 99 74 34 F2 AF  ..1.c..#..b.t4..
00000030: 99                                                .


In [71]:
m.pull(b'\0'*16), m

(b'',
 Hello ==> {'dx': 9.5367431640625e-07, 'dy': 9.5367431640625e-07, 'faces': [(1, 2, 5), (1, 5, 4), (4, 5, 8), (7, 4, 8), (0, 1, 4), (3, 0, 4), (3, 4, 7), (3, 7, 6)], 'level': 17, 'num_faces': 8, 'num_points': 9, 'ox': -0.6799468994140625, 'oy': 0.20928955078125, 'points': [(0, 0, 2.554255331688182e-07), (8, 0, 1.020640496562919e-07), (16, 0, 0.0), (0, 8, 1.016653300212056e-06), (8, 8, 4.082561844143129e-08), (16, 8, 0.0), (0, 16, 7.215168125185301e-07), (8, 16, 4.2490087004409816e-09), (16, 16, 0.0)], 'source_size': 0})

In [481]:
class Printable():
    cls = 'Printable'
    def __repr__(self):
        atts = {}
        for i in dir(self):
            if i[0] == '_' or i in ('main', 'sub', 'reg', 'cls'):
                continue
            att = self.__getattribute__(i)
            if inspect.ismethod(att):
                continue
            atts[i] = att
        return '%s ==> %s'%(self.cls,atts.__repr__())     

class Mesh(Printable):
    cls = 'Mesh'
    def __init__(self):
        self.points = []
        self.faces = []
        self.ox, self.oy, self.dx, self.dy, self.num_points, self.num_faces, self.level = 0,0,0,0,0,0,0
        pass
    def pull(self, buf):
        self.source_size, = struct.unpack('i', buf[:4])
        if self.source_size:
            rest = buf[4:]
        else:
            rest = buf[16:]
            return rest
        self.ox, self.oy, self.dx, self.dy, self.num_points, self.num_faces, self.level = struct.unpack('ddddiii', rest[:44])
        rest = rest[44:]
        for i in range(self.num_points):
#             self.points.append(struct.unpack('<BBf', rest[:6]))
            self.points.append((struct.unpack('<BBf', rest[:6]), rest[2:6]) )
            rest = rest[6:]
        for i in range(self.num_faces):
            self.faces.append(struct.unpack('<hhh', rest[:6]))
            rest = rest[6:]
        return rest
    
    def __repr__(self):
        atts = {}
        for i in dir(self):
            if i[0] == '_' or i in ('main', 'sub', 'reg'):
                continue
            att = self.__getattribute__(i)
            if inspect.ismethod(att):
                continue
            atts[i] = att
#         print(type(atts))
#         print(atts.keys())
#         return 'gogogo'
        return '%s ==> %s'%('Hello',atts.__repr__())


In [110]:
class KhDataHeader(Printable):
    cls = 'KhDataHeader'
    def __init__(self):
        pass
    def pull(self, buf):
          (self.magic_id, self.data_type_id, self.version, self.num_instances,
           self.data_instance_size, self.data_buffer_offset,self.data_buffer_size, self.meta_buffer_size
           ) = struct.unpack('<IIIiiiii', buf)
#     def __repr__(self):
#         atts = {}
#         for i in dir(self):
#             if i[0] == '_' or i in ('main', 'sub', 'reg'):
#                 continue
#             att = self.__getattribute__(i)
#             if inspect.ismethod(att):
#                 continue
#             atts[i] = att
#         return '%s ==> %s'%('Hello',atts.__repr__()) 
class KhQuantum(Printable):
    cls = 'KhQuantum'
    def pull(self, buf):
        (self.children, _, self.cnode_version, self.image_version, self.terrain_version, self.num_channels, _,
        self.type_offsset, self.version_offset, self.image_neiphors, self.image_provider, self.terrain_provider, _
        ) = struct.unpack('<BBHHHHHiiQBBH', buf)
        self._subnodes = [None, None, None, None]
        self._parrent = None
    def next_slot(self):
        for i in range(4):
            if 1<<i & self.children and (not self._subnodes[i]):
                return i
        return -1
    def next_node(self):
        ret = self
        while True:
            if ret.next_slot() >= 0:
                return ret
            ret = ret._parrent

In [401]:
def decode_qtpacket(buf):
    header = KhDataHeader()
    header.pull(buf[:32])
#     print(header)
#     print('='*50)
    items = []
    p = 32
    current_node = None
    root_node = None
    for i in range(header.num_instances):
        q = KhQuantum()
        q.pull(buf[p:p+32])
        p += 32
#         print(q)
#         print('-'*50)
        if i == 0:
            root_node = q
        else:
            cursor = current_node.next_node()
            q._parrent = cursor
            cursor._subnodes[cursor.next_slot()] = q
        current_node = q
    return header, root_node

In [154]:
r._subnodes[0]._subnodes[1]._subnodes[2]._subnodes[0]

KhQuantum ==> {'children': 64, 'cnode_version': 0, 'image_neiphors': 379, 'image_provider': 255, 'image_version': 861, 'num_channels': 0, 'terrain_provider': 0, 'terrain_version': 0, 'type_offsset': 0, 'version_offset': 0}

In [114]:
r = decode_qtpacket(ad)

KhDataHeader ==> {'data_buffer_offset': 10944, 'data_buffer_size': 1840, 'data_instance_size': 32, 'data_type_id': 1, 'magic_id': 32301, 'meta_buffer_size': 0, 'num_instances': 341, 'version': 2}
KhQuantum ==> {'children': 15, 'cnode_version': 0, 'image_neiphors': 0, 'image_provider': 0, 'image_version': 0, 'num_channels': 0, 'terrain_provider': 0, 'terrain_version': 0, 'type_offsset': 0, 'version_offset': 0}
--------------------------------------------------
KhQuantum ==> {'children': 239, 'cnode_version': 0, 'image_neiphors': 124, 'image_provider': 124, 'image_version': 844, 'num_channels': 1, 'terrain_provider': 0, 'terrain_version': 876, 'type_offsset': 0, 'version_offset': 2}
--------------------------------------------------
KhQuantum ==> {'children': 111, 'cnode_version': 0, 'image_neiphors': 353, 'image_provider': 255, 'image_version': 731, 'num_channels': 1, 'terrain_provider': 0, 'terrain_version': 0, 'type_offsset': 4, 'version_offset': 6}
-----------------------------------

In [214]:
# ad = http_get('https://kh.google.com/flatfile?q2-0-q.876')
# ad = http_get('https://kh.google.com/flatfile?q2-02102022001311212023-q.876')
ad = http_get('https://kh.google.com/flatfile?q2-0210202301212310-q.876')
# ad = http_get('http://172.16.234.12/db_1-v002/flatfile?q2-03012333-q.2')
# ad = http_get('http://172.16.234.13/db_1-v003/flatfile?q2-030132031330-q.3')
ad = decrypt(ad)
# hexdump.hexdump(ad)
ad = zlib.decompress(ad[8:])
r = decode_qtpacket(ad)

KhDataHeader ==> {'data_buffer_offset': 10944, 'data_buffer_size': 1392, 'data_instance_size': 32, 'data_type_id': 1, 'magic_id': 32301, 'meta_buffer_size': 0, 'num_instances': 341, 'version': 2}


In [472]:
# download_q2_2('0210202200120323')

In [458]:
qtp_top = None
import pathlib, os

def inorder_download(node, path_name, prefix, tree_path):
    flags = node.children
    if flags & 0x40: ## have image
        fn = '%s%s.%d.JFIF'%(prefix, tree_path, node.image_version)
        fn = os.path.join(path_name, fn)
        if not os.path.exists(fn):
            print('Download %s'%fn)
    if flags & 0x10: ## have subtree
        assert len(tree_path) == 4
        print('####################################')
        download_q2_2(prefix+tree_path, node.cnode_version)
    for i in range(4):
        subnode = node._subnodes[i]
        if subnode:
            inorder_download(subnode, path_name, prefix, '%s%d'%(tree_path,i))

def download_q2_2(levels, ver=876):
    p = blist_2_path(levels)
    if not os.path.exists(p):
        pathlib.Path(p).mkdir(parents=True, exist_ok=True)
    fn = '%s-q.%d' % (levels, ver)
    fn = os.path.join(p, fn)
    if not os.path.exists(fn):
        qp = hc.get('https://kh.google.com/flatfile?q2-%s-q.%d'%(levels,ver))
        qp = decrypt(qp)
        open(fn, 'wb').write(qp)
    else:
        qp = open(fn, 'rb').read()
    qp = zlib.decompress(qp[8:])
    header, qtp = decode_qtpacket(qp)
    inorder_download(qtp, p, levels, '')
    

def download_q2(levels, ver = 876):
#     assert(len(levels) == 16)
    ad = hc.get('https://kh.google.com/flatfile?q2-%s-q.%d'%(levels,ver))
    ad = decrypt(ad)
    ad = zlib.decompress(ad[8:])
    header, qtp = decode_qtpacket(ad)
    global qtp_top
    qtp_top = qtp
    go_down = False
    if header.num_instances == 341:
        a = qtp._subnodes[0]._subnodes[0]._subnodes[0]._subnodes[0].children &0x10
        if a:
            go_down = True
            for s1 in range(4):
                for s2 in range(4):
                    for s3 in range(4):
                        for s4 in range(4):
                            download_q2('%s%d%d%d%d'%(levels, s1, s2, s3, s4), ver)
    if not go_down:
        deep = -1
#         node = qtp
#         if node._subnodes[0] && node._subnodes[0].children &0x40:
#             deep = 1
#             node = node._subnodes[0]
#         
#         if qtp._subnodes[0]._subnodes[0]._subnodes[0]._subnodes[0].children &0x40:
#             deep = 4
#         elif qtp._subnodes[0]._subnodes[0]._subnodes[0].children &0x40:
#             deep = 3
#         elif qtp._subnodes[0]._subnodes[0].children &0x40:
#             deep = 2
#         elif qtp._subnodes[0].children &0x40:
#             deep = 1
        if header.num_instances == 341:
            deep = 4
        elif header.num_instances == 85:
            deep = 3
        elif header.num_instances == 21:
            deep = 2
        elif header.num_instances == 5:
            deep = 1
        download_tree_level(qtp, deep, levels)
        
def download_tree_level(root, deep, prefix):
    if not root:
        return
    if deep == 0:
#         assert(root.children & 0x40)
        if not (root.children & 0x40): return
        if os.path.exists('%s.%s.JFIF'%(prefix,root.image_version)):
#             print('skip %s.%s.JFIF'%(prefix,root.image_version))
            return
        url = 'https://kh.google.com/flatfile?f1-%s-i.%s'%(prefix, root.image_version)
#         print(url)
        content = hc.get(url)
        content = decrypt(content)
        open('%s.%s.JFIF'%(prefix,root.image_version), 'wb').write(content)
    else:
        for i in range(4):
            if root:
                download_tree_level(root._subnodes[i], deep-1, '%s%d'%(prefix,i))
        

In [191]:
levels = '0210202301212310'
ad = http_get('https://kh.google.com/flatfile?q2-%s-q.876'%levels)
hexdump.hexdump(ad)
ad = decrypt(ad)

00000000: 55 97 78 32 57 61 00 42  0F 81 80 F1 9D 9B 8F 3A  U.x2Wa.B.......:
00000010: 8F 46 1B 82 E7 14 92 7B  3D 0A B5 9F EA B4 99 60  .F.....{=......`
00000020: F6 39 A1 60 35 BB 54 70  7C 75 47 90 EE B8 7E F2  .9.`5.Tp|uG...~.
00000030: E4 D0 EA 9E 52 37 C7 53  33 7B 9F C9 0B EE 35 43  ....R7.S3{....5C
00000040: F1 00 56 9A 6D 15 0C 84  29 45 96 DF 3A 88 0A 41  ..V.m...)E..:..A
00000050: AD 9E DA D5 CD BC E6 5C  0E 60 8C A3 81 29 F2 B2  .......\.`...)..
00000060: 8A 18 45 F7 A7 3E 44 F2  59 98 EF DF 54 70 C4 AF  ..E..>D.Y...Tp..
00000070: 2B CE 7E BE FD D2 23 19  74 81 30 68 DE 96 9A C2  +.~...#.t.0h....
00000080: BB 86 36 15 28 4C 52 AF  C7 75 58 66 A4 05 86 C2  ..6.(LR..uXf....
00000090: DA 97 B1 A7 63 06 5C E6  30 56 1B D3 09 F8 C0 4A  ....c.\.0V.....J
000000A0: 4B 89 42 8D 57 0F 64 C0  84 49 03 F4 CF 80 71 FA  K.B.W.d..I....q.
000000B0: 49 F9 0E 2B 43 F2 3F 66  77 88 EB 8A 9F 2A 48 68  I..+C.?fw....*Hh
000000C0: FD 69 22 F2 4E 7E 16 25  96 99 3D 88 41 0B 46 79  .i".N~.%..=.A.Fy

In [189]:
len(ad)

2161

In [182]:
hexdump.hexdump(ad)

00000000: C4 68 54 09 24 05 59 12  32 3D 05 1D BB 9C A2 57  .hT.$.Y.2=.....W
00000010: F3 F5 11 7C 13 3F 8F 21  6C 0E F7 44 92 25 01 50  ...|.?.!l..D.%.P
00000020: A9 76 55 75 B2 ED 36 07  5C AD 04 66 38 DF 92 47  .vUu..6.\..f8..G
00000030: FF F8 F6 1A 84 5D 76 1F  00 52 70 7C 76 F6 71 71  .....]v..Rp|v.qq
00000040: 0B E4 AE 5F 9B 2E 08 5E  37 4B 85 7E 31 7C 79 6B  ..._...^7K.~1|yk
00000050: C0 4B 18 1A 0B 6E 0E 59  A7 41 03 3A B9 8B 1B 42  .K...n.Y.A.:...B
00000060: DB 7F DB 46 4B 20 87 4D  C6 1F 1D 45 B5 DD 21 11  ...FK .M...E..!.
00000070: D4 01 F3 50 33 56 97 72  61 4B 4D 70 47 A8 20 64  ...P3V.raKMpG. d
00000080: 9A B4 FE 78 B9 DD C4 27  EF 2B 87 1D 34 19 A4 5F  ...x...'.+..4.._
00000090: 0D 02 1C 41 54 98 65 28  5F 78 B7 04 E2 0C 4D 40  ...AT.e(_x....M@
000000A0: 83 4B BE 53 7F ED 35 4A  B7 EB 20 76 76 12 33 03  .K.S..5J.. vv.3.
000000B0: A2 CC 73 4B E2 DA 70 3A  70 0E 8A 26 86 56 D6 15  ..sK..p:p..&.V..
000000C0: C4 2D 32 16 3B F6 B1 2C  C0 B1 A3 21 97 52 95 6D  .-2.;..,...!.R.m

In [254]:
qtp_top._subnodes[0]._subnodes[0]._subnodes

[KhQuantum ==> {'children': 239, 'cnode_version': 0, 'image_neiphors': 379, 'image_provider': 255, 'image_version': 861, 'num_channels': 3, 'terrain_provider': 0, 'terrain_version': 854, 'type_offsset': 24, 'version_offset': 30},
 KhQuantum ==> {'children': 239, 'cnode_version': 0, 'image_neiphors': 379, 'image_provider': 255, 'image_version': 861, 'num_channels': 1, 'terrain_provider': 0, 'terrain_version': 854, 'type_offsset': 36, 'version_offset': 38},
 KhQuantum ==> {'children': 207, 'cnode_version': 0, 'image_neiphors': 379, 'image_provider': 255, 'image_version': 861, 'num_channels': 0, 'terrain_provider': 0, 'terrain_version': 854, 'type_offsset': 0, 'version_offset': 0},
 KhQuantum ==> {'children': 239, 'cnode_version': 0, 'image_neiphors': 379, 'image_provider': 255, 'image_version': 861, 'num_channels': 3, 'terrain_provider': 0, 'terrain_version': 854, 'type_offsset': 44, 'version_offset': 50}]

In [427]:
# download_q2('0210202301212022')
download_q2('0210202200120333')

KeyboardInterrupt: 

In [281]:
qtp_top

KhQuantum ==> {'children': 15, 'cnode_version': 0, 'image_neiphors': 0, 'image_provider': 0, 'image_version': 0, 'num_channels': 0, 'terrain_provider': 0, 'terrain_version': 0, 'type_offsset': 0, 'version_offset': 0}

In [293]:
def inorder(node, prefix=''):
    temp = '"%s"iv=%d,ip=%d,tv=%d,tp=%d,c=%d,' % (prefix, node.image_version,
        node.image_provider, node.terrain_version, node.terrain_provider, node.cnode_version)
    print(temp)
    for i in range(4):
        sub_node = node._subnodes[i]
        if sub_node:
            inorder(sub_node, '%s%d'%(prefix,i))

In [499]:
ad = hc.get('https://kh.google.com/flatfile?q2-0-q.876')
ad = decrypt(ad)
ad = zlib.decompress(ad[8:])

In [500]:
header, ro = decode_qtpacket(ad)

In [501]:
header

KhDataHeader ==> {'data_buffer_offset': 1984, 'data_buffer_size': 232, 'data_instance_size': 32, 'data_type_id': 1, 'magic_id': 32301, 'meta_buffer_size': 0, 'num_instances': 61, 'version': 2}

In [502]:
inorder(ro)

""iv=844,ip=124,tv=0,tp=0,c=0,
"0"iv=844,ip=124,tv=0,tp=0,c=0,
"00"iv=844,ip=0,tv=0,tp=0,c=0,
"002"iv=0,ip=0,tv=0,tp=0,c=876,
"003"iv=0,ip=0,tv=0,tp=0,c=876,
"01"iv=844,ip=0,tv=0,tp=0,c=0,
"012"iv=0,ip=0,tv=0,tp=0,c=876,
"02"iv=844,ip=124,tv=876,tp=0,c=0,
"020"iv=844,ip=124,tv=0,tp=0,c=876,
"021"iv=844,ip=124,tv=0,tp=0,c=876,
"022"iv=844,ip=124,tv=0,tp=0,c=876,
"023"iv=844,ip=255,tv=0,tp=0,c=876,
"03"iv=867,ip=124,tv=876,tp=0,c=0,
"030"iv=867,ip=124,tv=0,tp=0,c=876,
"031"iv=867,ip=124,tv=0,tp=0,c=876,
"032"iv=867,ip=124,tv=0,tp=0,c=876,
"033"iv=867,ip=124,tv=0,tp=0,c=876,
"1"iv=844,ip=124,tv=0,tp=0,c=0,
"10"iv=844,ip=0,tv=0,tp=0,c=0,
"102"iv=0,ip=0,tv=0,tp=0,c=876,
"103"iv=0,ip=0,tv=0,tp=0,c=876,
"11"iv=844,ip=0,tv=0,tp=0,c=0,
"112"iv=0,ip=0,tv=0,tp=0,c=876,
"113"iv=0,ip=0,tv=0,tp=0,c=876,
"12"iv=844,ip=124,tv=876,tp=0,c=0,
"120"iv=844,ip=124,tv=0,tp=0,c=876,
"121"iv=844,ip=124,tv=0,tp=0,c=876,
"122"iv=844,ip=124,tv=0,tp=0,c=876,
"123"iv=844,ip=124,tv=0,tp=0,c=876,
"13"iv=844,ip=124,tv

In [433]:
qtp_top._subnodes[0]._subnodes[2]

KhQuantum ==> {'children': 32, 'cnode_version': 0, 'image_neiphors': 0, 'image_provider': 0, 'image_version': 0, 'num_channels': 1, 'terrain_provider': 0, 'terrain_version': 0, 'type_offsset': 4, 'version_offset': 6}

In [78]:
def decode_all(buf):
    while buf:
        source_len, = struct.unpack('i', buf[:4])
        if len(buf) < source_len+4:
            hexdump.hexdump(buf)
            return
        m = Mesh()
        m.pull(buf[:source_len+4])
        buf = buf[source_len+4:]
        print('-'*50)
        print(m)
#         hexdump.hexdump(buf)

In [86]:
dhh = KhDataHeader()
dhh.pull(ac[:32])
dhh
# len(ac)

KhDataHeader ==> {'data_buffer_offset': 1472, 'data_buffer_size': 0, 'data_instance_size': 32, 'data_type_id': 1, 'magic_id': 32301, 'meta_buffer_size': 0, 'num_instances': 45, 'version': 2}

In [89]:
q = KhQuantum()
q.pull(ac[32:64])
q

KhQuantum ==> {'children': 79, 'cnode_version': 0, 'image_neiphors': 0, 'image_provider': 0, 'image_version': 1, 'num_channels': 0, 'terrain_provider': 0, 'terrain_version': 0, 'type_offsset': 0, 'version_offset': 0}

In [470]:
khWGS84EquatorialRadius = 6378137.0;
4.2490087004409816e-09 * 6378137

0.02710075960560454

In [57]:
m = Mesh()

m.pull(c[150:])

print(m)

Hello ==> {'dx': 9.5367431640625e-07, 'dy': 9.5367431640625e-07, 'faces': [(1, 2, 5), (1, 5, 4), (4, 5, 8), (7, 4, 8), (0, 1, 4), (3, 0, 4), (3, 4, 7), (3, 7, 6)], 'level': 17, 'num_faces': 8, 'num_points': 9, 'ox': -0.6799468994140625, 'oy': 0.20928955078125, 'points': [(0, 0, 2.554255331688182e-07), (8, 0, 1.020640496562919e-07), (16, 0, 0.0), (0, 8, 1.016653300212056e-06), (8, 8, 4.082561844143129e-08), (16, 8, 0.0), (0, 16, 7.215168125185301e-07), (8, 16, 4.2490087004409816e-09), (16, 16, 0.0)], 'source_size': 146}


In [61]:
struct.unpack('<d', c[4:12])

(-0.679962158203125,)

In [84]:
x, x = 1, 2; x

2

In [136]:
0.000092592592600 * 2399, 122.5450000-122.3228704, 0.000092592592600*40000000/360, 0.000277777777778*40000000/360

(0.22212962964740002,
 0.22212960000000237,
 10.288065844444446,
 30.864197530888887)

In [4]:
0.22212962964740002/ ( 360/(2<<22) ), 2<<9

(5175.9955230478245, 1024)

In [376]:
class Co_Range():
    def __init__(self, x1, x2, y1, y2):
        self.x1, self.x2, self.y1, self.y2 = x1, x2, y1, y2
    def __repr__(self):
        return ((self.x1, self.x2) , (self.y1, self.y2)).__repr__()
    def contains(self, point):
        x, y = point
        return x >= self.x1 and x < self.x2 and y >= self.y1 and y < self.y2

def blist_2_co_range(blist):
    ret = Co_Range(-180,180,-180,180)
    for i in blist[1:]: # first letter is always 0
        if i == '0':
            ret = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                           ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if i == '1':
            ret = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                           ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if i == '2':
            ret = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                           ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
        if i == '3':
            ret = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                           ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
    return ret

def point_2_blist(point, level):
    x, y = point
    ret = Co_Range(-180,180,-180,180)
    r = '0'
    for i in range(level - 1):
        ret0 = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                        ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if ret0.contains(point):
            r += '0'
            ret = ret0
            continue
        ret1 = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                       ret.y1, ret.y1 + (ret.y2-ret.y1)/2)
        if ret1.contains(point):
            r += '1'
            ret = ret1
            continue
        ret2 = Co_Range(ret.x1 + (ret.x2-ret.x1)/2, ret.x2,
                       ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
        if ret2.contains(point):
            r += '2'
            ret = ret2
            continue
        ret3 = Co_Range(ret.x1, ret.x1 + (ret.x2-ret.x1)/2,
                        ret.y1 + (ret.y2-ret.y1)/2, ret.y2)
        if ret3.contains(point):
            r += '3'
            ret = ret3
            continue
        assert(False)
    return r

class Blist():
    def __init__(self, blist):
        self.blist = blist
        self.co = blist_2_co_range(blist)
        co = self.co
        self.border_len = co.x2 - co.x1
        self.center = (co.x1+co.x2)/2, (co.y1+co.y2)/2
    def right(self):
        return point_2_blist((self.center[0]+self.border_len, self.center[1]), len(self.blist))
    def left(self):
        return point_2_blist((self.center[0]-self.border_len, self.center[1]), len(self.blist))
    def up(self):
        return point_2_blist((self.center[0], self.center[1]+self.border_len), len(self.blist))
    def down(self):
        return point_2_blist((self.center[0], self.center[1]-self.border_len), len(self.blist))
        
blist_2_co_range('02102023012120223333'), point_2_blist((119.4765, 31.4206),20)
b = Blist('02102023012120223333')
b.right(), b.left(), b.up(), b.down()

('02102023012120223332',
 '02102023012120232222',
 '02102023012123110000',
 '02102023012120223330')

In [385]:
import os
os.chdir('map')

In [436]:
def blist_2_path(blist):
    ret = []
    while blist:
        if len(blist) < 4:
            break
        ret.append(blist[:4])
        blist = blist[4:]
    return '/'.join(ret)

blist_2_path('123456789')

'1234/5678'

In [417]:
import shapefile
from shapely.geometry import Point, Polygon

class Boundary():
    def __init__(self, shape_file='CHN_adm0.shp'):
        sf = shapefile.Reader(shape_file)
        shapes = sf.shapes()
        self.poly = Polygon(shapes[0].points)
    def contain(self, point):
        p = Point(point[0], point[1])
        return p.within(self.poly)

# bd = Boundary()

def intersect(blist, poly):
    co_range = blist_2_co_range(blist)
    if Point(co_range.x1, co_range.y1).within(poly):
        return True
    if Point(co_range.x1, co_range.y2).within(poly):
        return True
    if Point(co_range.x2, co_range.y1).within(poly):
        return True
    if Point(co_range.x2, co_range.y2).within(poly):
        return True
    return False

def download_range(shape_file):
    sf = shapefile.Reader(shape_file)
    shape = sf.shapes()[0]
    poly = Polygon(shape.points)
    print(shape.bbox)
    bbox = shape.bbox
    left_bottom = bbox[0], bbox[1]
    left_bottom_blist = point_2_blist(left_bottom, 16)
    co_r = blist_2_co_range(left_bottom_blist)
    step = co_r.x2 - co_r.x1
    x_n = int((bbox[2] - bbox[0])/step) + 1
    y_n = int((bbox[3] - bbox[1])/step) + 1
#     blb = Blist(left_bottom_blist)
    left_blist = left_bottom_blist
    for i in range(x_n):
        p = left_blist
        for j in range(y_n):
            print(p)
            if intersect(p, poly):
                print('!!!!!!!')
                download_q2(p)
            p = Blist(p).right()
        left_blist = Blist(left_blist).up()
    
download_range('huangpu.shp')

[121.45222165840003, 31.189842571583682, 121.50580554608005, 31.246129886759967]
0210202200131220
0210202200131221
0210202200120330
0210202200120331
0210202200120320
0210202200120321
0210202200131223
0210202200131222
!!!!!!!
0210202200120333
!!!!!!!
0210202200120332
!!!!!!!
0210202200120323
!!!!!!!
0210202200120322
0210202200132110
0210202200132111
!!!!!!!
0210202200123000
!!!!!!!
0210202200123001
!!!!!!!
0210202200123010
!!!!!!!
0210202200123011
!!!!!!!
0210202200132113
!!!!!!!
0210202200132112
!!!!!!!
0210202200123003
!!!!!!!
0210202200123002
!!!!!!!
0210202200123013
!!!!!!!
0210202200123012
!!!!!!!
0210202200132120
!!!!!!!
0210202200132121
!!!!!!!
0210202200123030
!!!!!!!
0210202200123031
!!!!!!!
0210202200123020
!!!!!!!
0210202200123021
!!!!!!!


In [358]:
(119.47700500488281-119.476318359375)*40000000/360*16

1220.703125

In [379]:
int(3.7)

3

In [11]:
kDefaultKey = (
    b"\x45\xf4\xbd\x0b\x79\xe2\x6a\x45\x22\x05\x92\x2c"
    b"\x17\xcd\x06\x71\xf8\x49\x10\x46\x67\x51\x00\x42"
    b"\x25\xc6\xe8\x61\x2c\x66\x29\x08\xc6\x34\xdc\x6a"
    b"\x62\x25\x79\x0a\x77\x1d\x6d\x69\xd6\xf0\x9c\x6b"
    b"\x93\xa1\xbd\x4e\x75\xe0\x41\x04\x5b\xdf\x40\x56"
    b"\x0c\xd9\xbb\x72\x9b\x81\x7c\x10\x33\x53\xee\x4f"
    b"\x6c\xd4\x71\x05\xb0\x7b\xc0\x7f\x45\x03\x56\x5a"
    b"\xad\x77\x55\x65\x0b\x33\x92\x2a\xac\x19\x6c\x35"
    b"\x14\xc5\x1d\x30\x73\xf8\x33\x3e\x6d\x46\x38\x4a"
    b"\xb4\xdd\xf0\x2e\xdd\x17\x75\x16\xda\x8c\x44\x74"
    b"\x22\x06\xfa\x61\x22\x0c\x33\x22\x53\x6f\xaf\x39"
    b"\x44\x0b\x8c\x0e\x39\xd9\x39\x13\x4c\xb9\xbf\x7f"
    b"\xab\x5c\x8c\x50\x5f\x9f\x22\x75\x78\x1f\xe9\x07"
    b"\x71\x91\x68\x3b\xc1\xc4\x9b\x7f\xf0\x3c\x56\x71"
    b"\x48\x82\x05\x27\x55\x66\x59\x4e\x65\x1d\x98\x75"
    b"\xa3\x61\x46\x7d\x61\x3f\x15\x41\x00\x9f\x14\x06"
    b"\xd7\xb4\x34\x4d\xce\x13\x87\x46\xb0\x1a\xd5\x05"
    b"\x1c\xb8\x8a\x27\x7b\x8b\xdc\x2b\xbb\x4d\x67\x30"
    b"\xc8\xd1\xf6\x5c\x8f\x50\xfa\x5b\x2f\x46\x9b\x6e"
    b"\x35\x18\x2f\x27\x43\x2e\xeb\x0a\x0c\x5e\x10\x05"
    b"\x10\xa5\x73\x1b\x65\x34\xe5\x6c\x2e\x6a\x43\x27"
    b"\x63\x14\x23\x55\xa9\x3f\x71\x7b\x67\x43\x7d\x3a"
    b"\xaf\xcd\xe2\x54\x55\x9c\xfd\x4b\xc6\xe2\x9f\x2f"
    b"\x28\xed\xcb\x5c\xc6\x2d\x66\x07\x88\xa7\x3b\x2f"
    b"\x18\x2a\x22\x4e\x0e\xb0\x6b\x2e\xdd\x0d\x95\x7d"
    b"\x7d\x47\xba\x43\xb2\x11\xb2\x2b\x3e\x4d\xaa\x3e"
    b"\x7d\xe6\xce\x49\x89\xc6\xe6\x78\x0c\x61\x31\x05"
    b"\x2d\x01\xa4\x4f\xa5\x7e\x71\x20\x88\xec\x0d\x31"
    b"\xe8\x4e\x0b\x00\x6e\x50\x68\x7d\x17\x3d\x08\x0d"
    b"\x17\x95\xa6\x6e\xa3\x68\x97\x24\x5b\x6b\xf3\x17"
    b"\x23\xf3\xb6\x73\xb3\x0d\x0b\x40\xc0\x9f\xd8\x04"
    b"\x51\x5d\xfa\x1a\x17\x22\x2e\x15\x6a\xdf\x49\x00"
    b"\xb9\xa0\x77\x55\xc6\xef\x10\x6a\xbf\x7b\x47\x4c"
    b"\x7f\x83\x17\x05\xee\xdc\xdc\x46\x85\xa9\xad\x53"
    b"\x07\x2b\x53\x34\x06\x07\xff\x14\x94\x59\x19\x02"
    b"\xe4\x38\xe8\x31\x83\x4e\xb9\x58\x46\x6b\xcb\x2d"
    b"\x23\x86\x92\x70\x00\x35\x88\x22\xcf\x31\xb2\x26"
    b"\x2f\xe7\xc3\x75\x2d\x36\x2c\x72\x74\xb0\x23\x47"
    b"\xb7\xd3\xd1\x26\x16\x85\x37\x72\xe2\x00\x8c\x44"
    b"\xcf\x10\xda\x33\x2d\x1a\xde\x60\x86\x69\x23\x69"
    b"\x2a\x7c\xcd\x4b\x51\x0d\x95\x54\x39\x77\x2e\x29"
    b"\xea\x1b\xa6\x50\xa2\x6a\x8f\x6f\x50\x99\x5c\x3e"
    b"\x54\xfb\xef\x50\x5b\x0b\x07\x45\x17\x89\x6d\x28"
    b"\x13\x77\x37\x1d\xdb\x8e\x1e\x4a\x05\x66\x4a\x6f"
    b"\x99\x20\xe5\x70\xe2\xb9\x71\x7e\x0c\x6d\x49\x04"
    b"\x2d\x7a\xfe\x72\xc7\xf2\x59\x30\x8f\xbb\x02\x5d"
    b"\x73\xe5\xc9\x20\xea\x78\xec\x20\x90\xf0\x8a\x7f"
    b"\x42\x17\x7c\x47\x19\x60\xb0\x16\xbd\x26\xb7\x71"
    b"\xb6\xc7\x9f\x0e\xd1\x33\x82\x3d\xd3\xab\xee\x63"
    b"\x99\xc8\x2b\x53\xa0\x44\x5c\x71\x01\xc6\xcc\x44"
    b"\x1f\x32\x4f\x3c\xca\xc0\x29\x3d\x52\xd3\x61\x19"
    b"\x58\xa9\x7d\x65\xb4\xdc\xcf\x0d\xf4\x3d\xf1\x08"
    b"\xa9\x42\xda\x23\x09\xd8\xbf\x5e\x50\x49\xf8\x4d"
    b"\xc0\xcb\x47\x4c\x1c\x4f\xf7\x7b\x2b\xd8\x16\x18"
    b"\xc5\x31\x92\x3b\xb5\x6f\xdc\x6c\x0d\x92\x88\x16"
    b"\xd1\x9e\xdb\x3f\xe2\xe9\xda\x5f\xd4\x84\xe2\x46"
    b"\x61\x5a\xde\x1c\x55\xcf\xa4\x00\xbe\xfd\xce\x67"
    b"\xf1\x4a\x69\x1c\x97\xe6\x20\x48\xd8\x5d\x7f\x7e"
    b"\xae\x71\x20\x0e\x4e\xae\xc0\x56\xa9\x91\x01\x3c"
    b"\x82\x1d\x0f\x72\xe7\x76\xec\x29\x49\xd6\x5d\x2d"
    b"\x83\xe3\xdb\x36\x06\xa9\x3b\x66\x13\x97\x87\x6a"
    b"\xd5\xb6\x3d\x50\x5e\x52\xb9\x4b\xc7\x73\x57\x78"
    b"\xc9\xf4\x2e\x59\x07\x95\x93\x6f\xd0\x4b\x17\x57"
    b"\x19\x3e\x27\x27\xc7\x60\xdb\x3b\xed\x9a\x0e\x53"
    b"\x44\x16\x3e\x3f\x8d\x92\x6d\x77\xa2\x0a\xeb\x3f"
    b"\x52\xa8\xc6\x55\x5e\x31\x49\x37\x85\xf4\xc5\x1f"
    b"\x26\x2d\xa9\x1c\xbf\x8b\x27\x54\xda\xc3\x6a\x20"
    b"\xe5\x2a\x78\x04\xb0\xd6\x90\x70\x72\xaa\x8b\x68"
    b"\xbd\x88\xf7\x02\x5f\x48\xb1\x7e\xc0\x58\x4c\x3f"
    b"\x66\x1a\xf9\x3e\xe1\x65\xc0\x70\xa7\xcf\x38\x69"
    b"\xaf\xf0\x56\x6c\x64\x49\x9c\x27\xad\x78\x74\x4f"
    b"\xc2\x87\xde\x56\x39\x00\xda\x77\x0b\xcb\x2d\x1b"
    b"\x89\xfb\x35\x4f\x02\xf5\x08\x51\x13\x60\xc1\x0a"
    b"\x5a\x47\x4d\x26\x1c\x33\x30\x78\xda\xc0\x9c\x46"
    b"\x47\xe2\x5b\x79\x60\x49\x6e\x37\x67\x53\x0a\x3e"
    b"\xe9\xec\x46\x39\xb2\xf1\x34\x0d\xc6\x84\x53\x75"
    b"\x6e\xe1\x0c\x59\xd9\x1e\xde\x29\x85\x10\x7b\x49"
    b"\x49\xa5\x77\x79\xbe\x49\x56\x2e\x36\xe7\x0b\x3a"
    b"\xbb\x4f\x03\x62\x7b\xd2\x4d\x31\x95\x2f\xbd\x38"
    b"\x7b\xa8\x4f\x21\xe1\xec\x46\x70\x76\x95\x7d\x29"
    b"\x22\x78\x88\x0a\x90\xdd\x9d\x5c\xda\xde\x19\x51"
    b"\xcf\xf0\xfc\x59\x52\x65\x7c\x33\x13\xdf\xf3\x48"
    b"\xda\xbb\x2a\x75\xdb\x60\xb2\x02\x15\xd4\xfc\x19"
    b"\xed\x1b\xec\x7f\x35\xa8\xff\x28\x31\x07\x2d\x12"
    b"\xc8\xdc\x88\x46\x7c\x8a\x5b\x22")


In [14]:
def encrypt(raw, key = kDefaultKey):
    if len(raw) == 0 or len(key) == 0:
        return raw
    ret = bytearray(raw)
    dp = 0
    dpend = len(raw)
    op = 0
    kpstart = 0
    kpend = len(key)
    kp = 0
    off = 8
    end_2 = dpend & ~7
    
    while dp < end_2:
        off = (off + 8) % 24
        kp = kpstart + off
        
        while (dp < end_2) and (kp < kpend):
            for i in range(8):
                ret[op+i] = raw[dp+i] ^ key[kp+i]
            dp += 8
            op += 8
            kp += 24
            
    if dp < dpend:
        if kp >= kpend:
            off = (off + 8) % 24
            kp = kpstart + off
        while dp < dpend:
            ret[op] = raw[dp] ^ key[kp]
            dp += 1
            op += 1
            kp += 1
    return bytes(ret)

def decrypt(raw, key = kDefaultKey):
    return encrypt(raw, key)

In [308]:
resp = requests.request('GET', 'https://www.baidu.com', headers=Headers)

In [311]:
resp.status_code

200

In [307]:
http_get('http://localhost:7777/sdfjoei')

KeyboardInterrupt: 

In [438]:
import requests

Headers= {
'User-Agent':       'GoogleEarth/7.3.2.5776(Windows;Microsoft Windows (6.2.9200.0);en;kml:2.2;client:Pro;type:default)',
'Connection':       'Keep-Alive',
'Accept-Encoding':  'gzip, deflate',
'Accept-Language':  'zh-CN,en,*',
'Accept': None
}

Auth_Header = {
    'Accept': 'application/vnd.google-earth.kml+xml, application/vnd.google-earth.kmz, image/*, */*',
    'Content-Type': 'application/octet-stream',
    'User-Agent':'GoogleEarth/7.3.2.5776(Windows;Microsoft Windows (6.2.9200.0);en;kml:2.2;client:Pro;type:default)',
    'Content-Length': '49',
    'Connection': 'Keep-Alive',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,en,*'
}

def http_get(url):
    resp = requests.request('GET', url, headers=Headers)
    return resp.content

class HttpClient():
    def __init__(self):
        self.authed = False
        self.sk = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.addr = ('localhost', 7878)
        self.s = requests.session()
    def auth(self):
        buf = struct.pack('HHHH', random.randint(0,255*255),random.randint(0,255*255),
                          random.randint(0,255*255),random.randint(0,255*255))
        sk.sendto(buf, self.addr)
        auth_data,_ = sk.recvfrom(1024)
        resp = requests.request('POST', 'https://kh.google.com/geauth?ct=pro', data=auth_data,headers=Auth_Header)
        assert resp.status_code == 200
        raw = resp.content
        cookie_len, = struct.unpack('>i',raw[4:8])
        self.cookie = raw[8:8+cookie_len].decode('utf-8')
        self.authed = True
        Headers['Cookie'] = '$Version="0"; SessionId=%s; State=1'%self.cookie
    def get(self, url):
        if not self.authed:
            self.auth()
#         resp = requests.request('GET', url, headers=Headers)
        resp = self.s.request('GET', url, headers=Headers)
        if resp.status_code == 200:
            return resp.content
        if resp.status_code == 403:
            self.authed = False
            self.auth()
            resp = self.s.request('GET', url, headers=Headers)
            if resp.status_code == 200:
                return resp.content
            return None
        return None

import dbroot_v2_pb2
def parse_dbroot(raw):
    edr = dbroot_v2_pb2.EncryptedDbRootProto()
    edr.ParseFromString(raw)
    root = decrypt(edr.dbroot_data)
    root_pb = zlib.decompress(root[8:])
    dbroot = dbroot_v2_pb2.DbRootProto()
    dbroot.ParseFromString(root_pb)
    return dbroot

hc = HttpClient()

In [19]:
ab = open('/tmp/t/q.bin', 'rb').read()

In [38]:
test_q('http://172.16.234.13/db_1-v003/flatfile?q2-0-q.3')

00000000: 2D 7E 00 00 01 00 00 00  02 00 00 00 2D 00 00 00  -~..........-...
00000010: 20 00 00 00 C0 05 00 00  00 00 00 00 00 00 00 00   ...............
00000020: 4F 00 00 00 01 00 00 00  00 00 00 00 00 00 00 00  O...............
00000030: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000040: 4C 00 00 00 01 00 00 00  00 00 00 00 00 00 00 00  L...............
00000050: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000060: 4F 00 00 00 01 00 00 00  00 00 00 00 00 00 00 00  O...............
00000070: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000080: 50 00 03 00 01 00 00 00  00 00 00 00 00 00 00 00  P...............
00000090: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000A0: 50 00 03 00 01 00 00 00  00 00 00 00 00 00 00 00  P...............
000000B0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000C0: 50 00 03 00 01 00 00 00  00 00 00 00 00 00 00 00  P...............

In [416]:
hc = HttpClient()

In [346]:

def test_q(url):
#     ab = http_get(url)
    ab = hc.get(url)
    ab = decrypt(ab)
    ac = zlib.decompress(ab[8:])
    hexdump.hexdump(ac)

    
# ac = zlib.decompress(ab[8:])
# test_q('http://172.16.234.13/db_1-v003/flatfile?q2-0-q.3')
test_q('https://kh.google.com/flatfile?q2-0020-q.876')

00000000: 2D 7E 00 00 01 00 00 00  02 00 00 00 55 01 00 00  -~..........U...
00000010: 20 00 00 00 C0 2A 00 00  98 19 00 00 00 00 00 00   ....*..........
00000020: 0F 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000030: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000040: EF 00 00 00 4C 03 6C 03  01 00 00 00 00 00 00 00  ....L.l.........
00000050: 02 00 00 00 79 00 00 00  00 00 00 00 79 00 00 00  ....y.......y...
00000060: 6F 00 00 00 C4 02 00 00  01 00 00 00 04 00 00 00  o...............
00000070: 06 00 00 00 61 01 00 00  00 00 00 00 FF 00 00 00  ....a...........
00000080: EF 00 00 00 C4 02 68 03  02 00 00 00 08 00 00 00  ......h.........
00000090: 0C 00 00 00 61 01 00 00  00 00 00 00 FF 00 00 00  ....a...........
000000A0: 50 00 6C 03 C4 02 00 00  00 00 00 00 00 00 00 00  P.l.............
000000B0: 00 00 00 00 61 01 00 00  00 00 00 00 FF 00 00 00  ....a...........
000000C0: 50 00 6C 03 C4 02 00 00  00 00 00 00 00 00 00 00  P.l.............

In [34]:
# hexdump.hexdump(ac)
# open('/tmp/t/q.uncompressed', 'wb').write(ac)

In [493]:
root = http_get('http://172.16.234.12/db_1-v002/dbRoot.v5?hl=zh-hans&gl=cn&output=proto&cv=7.3.2.5815&ct=ec')
# root = http_get('https://kh.google.com/dbRoot.v5?hl=en&gl=cn&output=proto&cv=7.3.2.5776&ct=pro')

parse_dbroot(root)

imagery_present: true
terrain_present: true
end_snippet {
  model {
    radius: 6371.01
    negative_altitude_exponent_bias: 32
    compressed_negative_altitude_threshold: 1e-12
  }
  auth_server_url {
    value: ""
  }
  disable_authentication: true
  reverse_geocoder_url {
    value: ""
  }
  sky_database_is_available: false
  sky_database_url {
    value: ""
  }
  default_web_page_intl_url {
    value: ""
  }
  user_guide_intl_url {
    value: ""
  }
  support_center_intl_url {
    value: ""
  }
  business_listing_intl_url {
    value: ""
  }
  support_answer_intl_url {
    value: ""
  }
  support_topic_intl_url {
    value: ""
  }
  support_request_intl_url {
    value: ""
  }
  earth_intl_url {
    value: ""
  }
  add_content_url {
    value: ""
  }
  free_license_url {
    value: ""
  }
  pro_license_url {
    value: ""
  }
  diorama_description_url_base {
    value: ""
  }
  client_options {
    use_protobuf_quadtree_packets: false
    use_extended_copyright_ids: false
    preci

In [165]:
root = decrypt(edr.dbroot_data)
root
zlib.decompress(root[8:])

b'\x08\x01\x10\x01:\xe0\x03\n\x14\t\xf6(\\\x8f\x02\xe3\xb8@( 1\x11\xea-\x81\x99\x97q=\x12\x02\x12\x00\x18\x01:\x02\x12\x00H\x00R\x02\x12\x00Z\x02\x12\x00r\x02\x12\x00z\x02\x12\x00\x82\x01\x02\x12\x00\x8a\x01\x02\x12\x00\x92\x01\x02\x12\x00\x9a\x01\x02\x12\x00\xa2\x01\x02\x12\x00\xaa\x01\x02\x12\x00\xc2\x01\x02\x12\x00\xca\x01\x02\x12\x00\xe2\x01\x02\x12\x00\xf2\x01\x0e0\x008\x00B\x04\n\x002\x00P\x00z\x00\xfa\x01\x04r\x00z\x00\x82\x02\x04\n\x00\x10\x00\xc2\x02\x10\n\x02\x12\x00\x12\x02\x12\x00\x1a\x02\x12\x00"\x02\x12\x00\xca\x02\x10\n\x02\x12\x00\x12\x02\x12\x00\x1a\x02\x12\x00"\x02\x12\x00\xda\x02\x04\n\x02\x12\x00\xe2\x02\x04\n\x00\x12\x00\xf2\x02\x00\x82\x03\x02\x12\x00\x8a\x03\x02\x12\x00\x92\x03\x02\x12\x00\xb2\x03\xb8\x01\n@\n\x02\x12\x00\x12\r\x12\x0babout:blank"\r\x12\x0babout:blank*\r\x12\x0babout:blankR\r\x12\x0babout:blank\x1a*\x12(/earth/client/kmlrender/index_$[hl].html"*\x12(/earth/client/kmlrender/index_$[hl].html*\r\x12\x0babout:blank2\r\x12\x0babout:blank\xba\x03\x02\x

In [153]:
resp.content

b'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer><link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css><title>\xe7\x99\xbe\xe5\xba\xa6\xe4\xb8\x80\xe4\xb8\x8b\xef\xbc\x8c\xe4\xbd\xa0\xe5\xb0\xb1\xe7\x9f\xa5\xe9\x81\x93</title></head> <body link=#0000cc> <div id=wrapper> <div id=head> <div class=head_wrapper> <div class=s_form> <div class=s_form_wrapper> <div id=lg> <img hidefocus=true src=//www.baidu.com/img/bd_logo1.png width=270 height=129> </div> <form id=form name=f action=//www.baidu.com/s class=fm> <input type=hidden name=bdorz_come value=1> <input type=hidden name=ie value=utf-8> <input type=hidden name=f value=8> <input type=hidden name=rsv_bp value=1> <input type=hidden name=rsv_idx value=1> <input type=hidden name=tn value=baidu><span class="bg s_ipt_wr"><input id=kw name=wd class=s_ipt valu

In [496]:
(1.5696097932504927e-07 * khWGS84EquatorialRadius, 0.00015868755872361362*khWGS84EquatorialRadius,
 0.0006082655163481832*6371010, khWGS84EquatorialRadius)

(1.0011186297893317, 1012.1309897347528, 3875.2656873094384, 6378137.0)

In [498]:
ab = http_get('http://172.16.234.12/t4db-v005/flatfile?f1c-03013203021221321-t.5')
ac = decrypt(ab)
# hexdump.hexdump(ac)
ad = zlib.decompress(ac[8:])
decode_all(ad)

--------------------------------------------------
Hello ==> {'cls': 'Mesh', 'dx': 9.5367431640625e-07, 'dy': 9.5367431640625e-07, 'faces': [(1, 2, 5), (4, 5, 8), (4, 8, 7), (0, 1, 5), (0, 5, 4), (0, 4, 3), (3, 4, 6), (6, 4, 7)], 'level': 17, 'num_faces': 8, 'num_points': 9, 'ox': -0.679840087890625, 'oy': 0.20867919921875, 'points': [((0, 0, 0.0), b'\x00\x00\x00\x00'), ((8, 0, 0.0), b'\x00\x00\x00\x00'), ((16, 0, 0.0), b'\x00\x00\x00\x00'), ((0, 8, 0.00010263390868203714), b')=\xd78'), ((8, 8, 0.00010296256368746981), b'\x9b\xed\xd78'), ((16, 8, 0.00010329121141694486), b'\x0c\x9e\xd88'), ((0, 16, 0.0002188198996009305), b'\x05se9'), ((8, 16, 0.00021952282986603677), b'\xb6/f9'), ((16, 16, 0.0002202257892349735), b'i\xecf9')], 'source_size': 146}
--------------------------------------------------
Hello ==> {'cls': 'Mesh', 'dx': 9.5367431640625e-07, 'dy': 9.5367431640625e-07, 'faces': [(1, 2, 5), (1, 5, 4), (4, 5, 8), (4, 8, 7), (0, 1, 4), (0, 4, 3), (3, 4, 6), (6, 4, 7)], 'level': 17,

In [484]:
struct.unpack('i', b'^e&9')

(958817630,)

In [132]:
ac = decrypt(ab)
# hexdump.hexdump(ac)
ad = zlib.decompress(ac[8:])
decode_all(ad)

--------------------------------------------------
Hello ==> {'dx': 3.0517578125e-05, 'dy': 3.0517578125e-05, 'faces': [(1, 2, 4), (4, 2, 5), (4, 5, 7), (7, 5, 8), (0, 1, 4), (0, 4, 3), (3, 4, 7), (3, 7, 6)], 'level': 9, 'num_faces': 8, 'num_points': 9, 'ox': 0.46875, 'oy': 0.1484375, 'points': [(0, 0, 1.2085995876986999e-05), (64, 0, 1.2085995876986999e-05), (128, 0, 1.1144229574711062e-05), (0, 64, 1.3184722774894908e-05), (64, 64, 1.38125669764122e-05), (128, 64, 1.6951786165009253e-05), (0, 128, 4.8187022912316024e-05), (64, 128, 7.42425472708419e-05), (128, 128, 0.00011568024638108909)], 'source_size': 146}
--------------------------------------------------
Hello ==> {'dx': 3.0517578125e-05, 'dy': 3.0517578125e-05, 'faces': [(1, 2, 4), (4, 2, 5), (4, 5, 8), (4, 8, 7), (0, 1, 4), (3, 0, 4), (6, 3, 4), (6, 4, 7)], 'level': 9, 'num_faces': 8, 'num_points': 9, 'ox': 0.47265625, 'oy': 0.1484375, 'points': [(0, 0, 1.1144229574711062e-05), (64, 0, 1.1144229574711062e-05), (128, 0, 1.1772

In [137]:
uu = b"E\364\275\013y\342jE\"\005\222,\027\315\006q\370I\020FgQ\000B%\306\350a,f)\010\3064\334jb%y\nw\035mi\326\360\234k\223\241\275Nu\340A\004[\337@V\014\331\273r\233\201|\0203S\356Ol\324q\005\260{\300\177E\003VZ\255wUe\0133\222*\254\031l5\024\305\0350s\3703>mF8J\264\335\360.\335\027u\026\332\214Dt\"\006\372a\"\0143\"So\2579D\013\214\0169\3319\023L\271\277\177\253\\\214P_\237\"ux\037\351\007q\221h;\301\304\233\177\360<VqH\202\005\'UfYNe\035\230u\243aF}a?\025A\000\237\024\006\327\2644M\316\023\207F\260\032\325\005\034\270\212\'{\213\334+\273Mg0\310\321\366\\\217P\372[/F\233n5\030/\'C.\353\n\014^\020\005\020\245s\033e4\345l.jC\'c\024#U\251?q{gC}:\257\315\342TU\234\375K\306\342\237/(\355\313\\\306-f\007\210\247;/\030*\"N\016\260k.\335\r\225}}G\272C\262\021\262+>M\252>}\346\316I\211\306\346x\014a1\005-\001\244O\245~q \210\354\r1\350N\013\000nPh}\027=\010\r\027\225\246n\243h\227$[k\363\027#\363\266s\263\r\013@\300\237\330\004Q]\372\032\027\".\025j\337I\000\271\240wU\306\357\020j\277{GL\177\203\027\005\356\334\334F\205\251\255S\007+S4\006\007\377\024\224Y\031\002\3448\3501\203N\271XFk\313-#\206\222p\0005\210\"\3171\262&/\347\303u-6,rt\260#G\267\323\321&\026\2057r\342\000\214D\317\020\3323-\032\336`\206i#i*|\315KQ\r\225T9w.)\352\033\246P\242j\217oP\231\\>T\373\357P[\013\007E\027\211m(\023w7\035\333\216\036J\005fJo\231 \345p\342\271q~\014mI\004-z\376r\307\362Y0\217\273\002]s\345\311 \352x\354 \220\360\212\177B\027|G\031`\260\026\275&\267q\266\307\237\016\3213\202=\323\253\356c\231\310+S\240D\\q\001\306\314D\0372O<\312\300)=R\323a\031X\251}e\264\334\317\r\364=\361\010\251B\332#\t\330\277^PI\370M\300\313GL\034O\367{+\330\026\030\3051\222;\265o\334l\r\222\210\026\321\236\333?\342\351\332_\324\204\342FaZ\336\034U\317\244\000\276\375\316g\361Ji\034\227\346 H\330]\177~\256q \016N\256\300V\251\221\001<\202\035\017r\347v\354)I\326]-\203\343\3336\006\251;f\023\227\207j\325\266=P^R\271K\307sWx\311\364.Y\007\225\223o\320K\027W\031>\'\'\307`\333;\355\232\016SD\026>?\215\222mw\242\n\353?R\250\306U^1I7\205\364\305\037&-\251\034\277\213\'T\332\303j \345*x\004\260\326\220pr\252\213h\275\210\367\002_H\261~\300XL?f\032\371>\341e\300p\247\3178i\257\360VldI\234\'\255xtO\302\207\336V9\000\332w\013\313-\033\211\3735O\002\365\010Q\023`\301\nZGM&\03430x\332\300\234FG\342[y`In7gS\n>\351\354F9\262\3614\r\306\204Sun\341\014Y\331\036\336)\205\020{II\245wy\276IV.6\347\013:\273O\003b{\322M1\225/\2758{\250O!\341\354Fpv\225})\"x\210\n\220\335\235\\\332\336\031Q\317\360\374YRe|3\023\337\363H\332\273*u\333`\262\002\025\324\374\031\355\033\354\1775\250\377(1\007-\022\310\334\210F|\212[\""
dbroot_data: "U\227x2\212P\000B\017\201\216\t\302\220(\021[MY\322\312\036\315\351\224\332c$\253\024iv\034@\317\233\351+S\2563\3500\"\004\211\345~\210\313\035_\320%S\265$\177ID\304\322\230F\331\200\230g\271\\\324p\207\014\373\273\006\032\014\007\270\2301\277DS\214\263W\342\346\306\nf\262\2529\320:\n\276\354 \337\262]:1\030\206o3)`\24789y\023\025\275 w2o\033\003 SD\236\361\005#\354U\001\272\271\325\"\t\243O+\017-68\2622\334\265m\356\022\277\351V\230\250\033\250I\265\016\312f\376k:\254`M\353\035r\242\340q\'\347+p8mM\302y\204%}]\240\300\275\325z\336\271X\000S\257 \201e\334>\262\273[r\311\271\n\177x\311\364C\325p\3354\242\242\022\354\006\362.^\315\2547q\322\177c\337\332\362k%e\337\000\007\254+\261\364v\341\034\222\003\351\347\265\366\247\024\266\371\260i\266\251\200q\314E>\261k\037\374k3\017\312F\002\217\341y\035\330k|\270\247\2557\202\344\004\017\306E]\036|"


In [150]:
cu = b'U\x97x2\x8aP\x00B\x0f\x81\x8e\t\xc2\x90(\x11[MY\xd2\xca\x1e\xcd\xe9\x94\xdac$\xab\x14iv\x1c@\xcf\x9b\xe9+S\xae3\xe80"\x04\x89\xe5~\x88\xcb\x1d_\xd0%S\xb5$\x7fID\xc4\xd2\x98F\xd9\x80\x98g\xb9\\\xd4p\x87\x0c\xfb\xbb\x06\x1a\x0c\x07\xb8\x981\xbfDS\x8c\xb3W\xe2\xe6\xc6\nf\xb2\xaa9\xd0:\n\xbe\xec \xdf\xb2]:1\x18\x86o3)`\xa789y\x13\x15\xbd w2o\x1b\x03 SD\x9e\xf1\x05#\xecU\x01\xba\xb9\xd5"\t\xa3O+\x0f-68\xb22\xdc\xb5m\xee\x12\xbf\xe9V\x98\xa8\x1b\xa8I\xb5\x0e\xcaf\xfek:\xac`M\xeb\x1dr\xa2\xe0q\'\xe7+p8mM\xc2y\x84%}]\xa0\xc0\xbd\xd5z\xde\xb9X\x00S\xaf \x81e\xdc>\xb2\xbb[r\xc9\xb9\n\x7fx\xc9\xf4C\xd5p\xdd4\xa2\xa2\x12\xec\x06\xf2.^\xcd\xac7q\xd2\x7fc\xdf\xda\xf2k%e\xdf\x00\x07\xac+\xb1\xf4v\xe1\x1c\x92\x03\xe9\xe7\xb5\xf6\xa7\x14\xb6\xf9\xb0i\xb6\xa9\x80q\xccE>\xb1k\x1f\xfck3\x0f\xcaF\x02\x8f\xe1y\x1d\xd8k|\xb8\xa7\xad7\x82\xe4\x04\x0f\xc6E]\x1e|'
pp = decrypt(cu)
# hexdump.hexdump(pp)
qq = zlib.decompress(pp[8:])
qq

b'\x08\x01\x10\x01:\xe0\x03\n\x14\t\xf6(\\\x8f\x02\xe3\xb8@( 1\x11\xea-\x81\x99\x97q=\x12\x02\x12\x00\x18\x01:\x02\x12\x00H\x00R\x02\x12\x00Z\x02\x12\x00r\x02\x12\x00z\x02\x12\x00\x82\x01\x02\x12\x00\x8a\x01\x02\x12\x00\x92\x01\x02\x12\x00\x9a\x01\x02\x12\x00\xa2\x01\x02\x12\x00\xaa\x01\x02\x12\x00\xc2\x01\x02\x12\x00\xca\x01\x02\x12\x00\xe2\x01\x02\x12\x00\xf2\x01\x0e0\x008\x00B\x04\n\x002\x00P\x00z\x00\xfa\x01\x04r\x00z\x00\x82\x02\x04\n\x00\x10\x00\xc2\x02\x10\n\x02\x12\x00\x12\x02\x12\x00\x1a\x02\x12\x00"\x02\x12\x00\xca\x02\x10\n\x02\x12\x00\x12\x02\x12\x00\x1a\x02\x12\x00"\x02\x12\x00\xda\x02\x04\n\x02\x12\x00\xe2\x02\x04\n\x00\x12\x00\xf2\x02\x00\x82\x03\x02\x12\x00\x8a\x03\x02\x12\x00\x92\x03\x02\x12\x00\xb2\x03\xb8\x01\n@\n\x02\x12\x00\x12\r\x12\x0babout:blank"\r\x12\x0babout:blank*\r\x12\x0babout:blankR\r\x12\x0babout:blank\x1a*\x12(/earth/client/kmlrender/index_$[hl].html"*\x12(/earth/client/kmlrender/index_$[hl].html*\r\x12\x0babout:blank2\r\x12\x0babout:blank\xba\x03\x02\x

In [141]:
hexdump.hexdump(uu)

00000000: 45 F4 BD 0B 79 E2 6A 45  22 05 92 2C 17 CD 06 71  E...y.jE"..,...q
00000010: F8 49 10 46 67 51 00 42  25 C6 E8 61 2C 66 29 08  .I.FgQ.B%..a,f).
00000020: C6 34 DC 6A 62 25 79 0A  77 1D 6D 69 D6 F0 9C 6B  .4.jb%y.w.mi...k
00000030: 93 A1 BD 4E 75 E0 41 04  5B DF 40 56 0C D9 BB 72  ...Nu.A.[.@V...r
00000040: 9B 81 7C 10 33 53 EE 4F  6C D4 71 05 B0 7B C0 7F  ..|.3S.Ol.q..{..
00000050: 45 03 56 5A AD 77 55 65  0B 33 92 2A AC 19 6C 35  E.VZ.wUe.3.*..l5
00000060: 14 C5 1D 30 73 F8 33 3E  6D 46 38 4A B4 DD F0 2E  ...0s.3>mF8J....
00000070: DD 17 75 16 DA 8C 44 74  22 06 FA 61 22 0C 33 22  ..u...Dt"..a".3"
00000080: 53 6F AF 39 44 0B 8C 0E  39 D9 39 13 4C B9 BF 7F  So.9D...9.9.L...
00000090: AB 5C 8C 50 5F 9F 22 75  78 1F E9 07 71 91 68 3B  .\.P_."ux...q.h;
000000A0: C1 C4 9B 7F F0 3C 56 71  48 82 05 27 55 66 59 4E  .....<VqH..'UfYN
000000B0: 65 1D 98 75 A3 61 46 7D  61 3F 15 41 00 9F 14 06  e..u.aF}a?.A....
000000C0: D7 B4 34 4D CE 13 87 46  B0 1A D5 05 1C B8 8A 27  ..4M...F.......'

In [506]:
0xc0, len(abc), len(zlib.decompress(abc[8:]))

(192, 46, 192)

In [503]:
abc = open('/root/py/m3/0210/2021/3233/1132/0210202132331132-q.876', 'rb').read()

hexdump.hexdump(abc)

00000000: AD DE 68 74 C0 00 00 00  78 9C D3 AD 63 60 60 64  ..ht....x...c``d
00000010: 60 60 60 02 62 56 20 56  00 E2 03 0C 08 C0 CF 80  ```.bV V........
00000020: 1F 34 80 C9 30 E6 81 92  07 00 2D 8C 05 07        .4..0.....-...
